# CO_2 and Weather

## Formalia:

What are we trying to figure out

## A1 Part 1: Visualizing patterns in the data

In this sub-assignment, we recreate some of the plots from Week 2. We look only at the focus-crimes.

In [ ]:
EU = ['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden'] 

For each set of plots below, describe the plots (as you would in the figure text in a report or paper), and pick a few aspects that stand out to you and comment on those (we provided a couple of ideas during class but it's OK to add more).

* First create the week-day plots, the months, the 24-hour cycle, and the 168 hours of the week as barcharts (Week 2, Part 2).
* Next choose a crime type that you like and create a calendar plot (Week 2, Part 4). Don't forget to comment on patterns you observe.
* Finally, choose a different crime type and create a polar plot of its 24hour cycle (Week 2, Part 4). Again, don't forget to comment as you would in a figure text.
* Write a short reflection (max 400 words, shorter is OK) on how you've used LLMs to solve these assignments (or if you didn't use LLMs, explain why).

## Answer for A1 Part 1


In [ ]:
# Import libraries
from math import ceil
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import geojson
import json
#Only two digit after the decimal point
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Importting libraries for data manipulation and visualization
import matplotlib.pyplot as plt
import calendar 
import calplot

from urllib.request import urlopen



import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning

In [ ]:
# read the data and create two dataframes, one for the EU and one for the rest of the world and also one for each land and year with their CO2 emission
df = pd.read_csv('CO2_Emissions_1960-2018.xls')
#replace NaN with 0
df = df.fillna(0)  
#replace inf with 0
df = df.replace([np.inf, -np.inf], 0)

# if Slovak Republic then replace with Slovakia
df.replace(to_replace='Slovak Republic', value='Slovakia', inplace=True)
# Czech Republic to Czechia
df.replace(to_replace='Czech Republic', value = 'Czechia', inplace=True)

df.head()

In [ ]:
df_EU = df.copy()
df_EU = df_EU[df_EU['Country Name'].isin(EU)]


In [ ]:
# Some basis statistics using stats for EU CO2 emission
df_EU.describe()


In [ ]:
# Some basis statistics using stats for EU CO2 emission
decribe_stat = df_EU.describe()
# plot the df_EU.describe() each stat in a subplot
fig, ax = plt.subplots(2, 3, figsize=(27, 12))
fig.suptitle('Statistics for EU CO2 Emission')
decribe_stat.loc['mean'].plot(kind='bar', ax=ax[0, 0], title='Mean',grid=True, yticks=range(0, 9))
decribe_stat.loc['std'].plot(kind='bar', ax=ax[0, 1], title='Standard Deviation',grid=True, yticks=range(0, 9))
decribe_stat.loc['min'].plot(kind='bar', ax=ax[0, 2], title='Min',grid=True, yticks=range(0, 5))
decribe_stat.loc['25%'].plot(kind='bar', ax=ax[1, 0], title='25%',grid=True, yticks=range(0, 7))
decribe_stat.loc['50%'].plot(kind='bar', ax=ax[1, 1], title='50%',grid=True, yticks=range(0, 9))
decribe_stat.loc['75%'].plot(kind='bar', ax=ax[1, 2], title='75%',grid=True, yticks=range(0, 12))

plt.show()
#save the figure
fig.savefig('EU_CO2_Emission_Statistics.png')


In [ ]:
df_EU_yearly = df.copy()
df_EU_yearly = df_EU.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')


In [ ]:
for index, country in enumerate(EU):
    y_min, y_max = 0, df_EU_yearly[df_EU_yearly['Country Name'] == country]['CO2 Emission'].max()   
    print(f'{country} --> y_min: {y_min}, y_max: {y_max}')

In [ ]:
# First first row is the year and first coloumn is the country name, make a plot where the x-axis is the year and the y-axis is the CO2 emission and make it so theres a subplot for each country
col = 3
rows = math.ceil(len(EU) / col)


fig, ax = plt.subplots(rows, col, sharex=True, figsize=(24, 15))

for index, country in enumerate(EU):
    plt.subplot(rows, col, index+1)
    plt.title(country)#, x = 0.1, y=0.7, pad=20) 
    plt.xlabel('Year')
    plt.ylabel('CO2 Emission')
    y_min, y_max = 0, df_EU_yearly[df_EU_yearly['Country Name'] == country]['CO2 Emission'].max()   
    #print(f'{country} --> y_min: {y_min}, y_max: {y_max}')
    plt.ylim([y_min, y_max])
    y_interval = int((y_max - y_min) / 5) #if y_max != 'nan' else 0
    plt.yticks(range(0, int(y_max), y_interval))
    plt.xticks(rotation=90)
    # Bar chart for each country with the CO2 emission for each year
    plt.bar(df_EU_yearly[df_EU_yearly['Country Name'] == country]['Year'], df_EU_yearly[df_EU_yearly['Country Name'] == country]['CO2 Emission'])
   
    plt.tight_layout()
    plt.title('CO2 Emission for ' + country)
plt.tight_layout()

# save to html
plt.savefig('CO2_Emission_EU.png')
fig.suptitle('CO2 Emission for each country in the EU distribuated through the years 1960-2018', fontsize=16, y=1.05)
plt.show()



In [ ]:
# First first row is the year and first coloumn is the country name, make a plot where the x-axis is the year and the y-axis is the CO2 emission and make it so theres a subplot for each country
fig = px.line(df_EU_yearly, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for each country in the EU')
fig.show()
# save interactive figure as html
fig.write_html("CO2_Emission_EU.html")


In [ ]:
# Timeserie with world co2 emission
df_world = df.copy()
df_world = df_world[~df_world['Country Name'].isin(EU)]
df_world_yearly = df_world.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')
#plot the world co2 emission
fig = px.line(df_world_yearly, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for each country in the world')
fig.show()
# save interactive figure as html
fig.write_html("CO2_Emission_World.html")

In [ ]:
df['max_year'] = df.select_dtypes(include=[np.number]).max(axis=1)
df['year_max'] = df.select_dtypes(include=[np.number]).idxmax(axis=1)
#df['min_year'] = df.min(axis=1)

for i in range(4, 60):
    df.loc[df['year_max'] == df.columns[i], 'year_max'] = df.columns[i]

# plotly bar plot witg country name, max_year and year_max
fig = go.Figure(data=[go.Table(
    header=dict(values=['Country Name', 'Max CO2 Emission', 'Year of Max CO2 Emission']),
    cells=dict(values=[df['Country Name'], df['max_year'], df['year_max']])
)])
fig.update_layout(title='Max CO2 Emission and Year of Max CO2 Emission for countries')
#fig.write_html("Max_CO2_Emission_and_Year_of_Max_CO2_Emission_for_countries.html")
fig.show()
#save as html
fig.write_html("Max_CO2_Emission_and_Year_of_Max_CO2_Emission_for_countries.html")


In [ ]:
df_EU['max_year'] = df_EU.select_dtypes(include=[np.number]).max(axis=1)
df_EU['min_year'] = df_EU.select_dtypes(include=[np.number]).min(axis=1)
df_EU['year_max'] = df_EU.select_dtypes(include=[np.number]).idxmax(axis=1)
df_EU['year_min'] = df_EU.select_dtypes(include=[np.number]).idxmin(axis=1)


# plotly bar plot with country name, max_year, min_year, year_max and year_min
fig = go.Figure(data=[go.Table(
    header=dict(values=['Country Name', 'Max CO2 Emission', 'Year of Max CO2 Emission', 'Min CO2 Emission', 'Year of Min CO2 Emission']),
    cells=dict(values=[df_EU['Country Name'], df_EU['max_year'], df_EU['year_max'], df_EU['min_year'], df_EU['year_min']])
)])
fig.update_layout(title='Max and Min CO2 Emission and Year of Max and Min CO2 Emission for EU countries')
fig.write_html("List_Max_and_Min_CO2_Emission_and_Year_of_Max_and_Min_CO2_Emission_for_EU_countries.html")
fig.show()
print(f'First part')

In [ ]:
# plotly bar chart with country name, max_year, year_max, min_year and year_min
fig = go.Figure(data=[ go.Bar(name='Min CO2 Emission', x=df_EU['Country Name'], y=df_EU['min_year'], text=df_EU['year_min'], textposition='auto'),
                      go.Bar(name='Max CO2 Emission', x=df_EU['Country Name'], y=df_EU['max_year'], text=df_EU['year_max'], textposition='auto')])
fig.update_layout(title='Max and Min CO2 Emission and Year of Max and Min CO2 Emission for EU countries', barmode='group')
fig.write_html("Bar_chart_Max_and_Min_CO2_Emission_and_Year_of_Max_and_Min_CO2_Emission_for_EU_countries.html")
fig.show()

## A1 Part 3: Visualizing geodata using Plotly

In this sub-assignment I want you to create a choropleth map of San Francisco, showing where it's safest to leave your car on Sundays, following the instructions from Week 3.

When you're done, reflect on the following questions.

* Based on your map and analysis, where should you park the car for it to be safest on a Sunday? And where's the worst place?
* Using visualizatios can help us uncover powerful data-patterns. However, when designing visualizations, we need to be aware of several illusions that can lead viewers to misinterpret the data we are showing (i.e. perceptual errors):
   - Try to change the range of data-values in the plot above. Is there a way to make the difference between district less evident?
   - Why do you think perceptual errors are a problem? Try to think of a few examples. 

In [ ]:
df_EU_yearly.head()

In [ ]:


with  open('europe.geojson') as response: #### Using the geojson of lat and lon of San Fransisco
    counties = geojson.load(response) 

fig = px.choropleth(df_EU_yearly, 
                           geojson=counties, 
                           locations='Country Name', 
                           featureidkey='properties.NAME',
                           color='CO2 Emission',
                           color_continuous_scale= "turbo", ## ????
                           hover_name = 'Country Name',
                           hover_data=  ['Year', 'CO2 Emission'],
                           animation_frame='Year',
                           #locationmode='geojson-id',
                           range_color=(0, 40),
                           #mapbox_style="carto-positron",
                           scope='europe',
                           center = {"lat":52.52000660, "lon": 13.404954007},
                           #opacity=1,
                           #labels={'CO2 Emission':'CO2 Emission'}
                           width=1000, height=800,
                           title = 'CO2 for EU from 1960-2018'
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
#save as html
fig.write_html("CO2_Emission_EU_Map_Online.html", include_plotlyjs='cdn')
"""'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']."""